<h1>Problem 1:</h1>
Write a function that reads timeseries pricing data from a file into a pandas dataframe and then groups the data as follows:
<li>The arguments to the function are the filename and a threshold number
<li>The function reads the data in the file and creates a new column "pct_change" with the one day percent change
<li>Then groups the data into four categories:
<ul>
<li>"High+" if the percent change is greater than the threshold 
<li>"Low+" if the percent change is zero or positive and less than or equal to the threshold% 
<li>"Low-" if the percent change is negative but greater than or equal to -1 * the threshold
<li>"High-" if the percent change is less than -1 * the threshold
</ul>
<li>The function should return a dataframe that contains three columns (count, mean, stdev) and four index values (High+, High-, Low+, Low-)
    <p><b>Note: </b>we have to deal with nan percent changes. For the purposes of this assignment, you can either classify it as a High- (in which case you'll get the results below) or classify it as "unknown" (your results for the High- row below will be slightly different).<p>
For the sample data your function should return a dataframe with the following values for a threshold of 1.0:

<pre>


        count	mean	stdev
High+	698	32.270473	24.973644
High-	520	29.824000	22.791628
Low+	769	29.234083	19.201473
Low-	719	29.193588	18.501548
</pre>

    

<h3>read_csv</h3>
The pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">read_csv</a> function reads data from a delimited file into a pandas dataframe.



<h2>Sample data</h2>
Use the attached AAPL.csv file

In [10]:
def change_groups(datafile,threshold):
    import pandas as pd
    import numpy as np

    def change_group(df,ind,col,threshold):
        val = df[col][ind]
        if val > threshold:
            return "High+"
        elif val >= 0:
            return "Low+"
        elif val >-1.0*threshold:
            return "Low-"
        else:
            return "High-"

    datafile = "AAPL.csv"
    df = pd.read_csv(datafile,header=0,na_values='null')
    df['pct_changes']=(df['Close'].pct_change())*100

    groups = df.groupby(lambda x: change_group(df,x,'pct_changes',threshold))
    count = groups.count()['Date']
    mean = groups.mean()['pct_changes']
    stdev = groups.std()['pct_changes']
    return pd.DataFrame({'count':count,'mean':mean,'stdev':stdev})


In [11]:
change_groups("AAPL.csv",1.0)

,count,mean,stdev
High+,698,2.128898,1.323162
High-,520,-2.264811,1.439012
Low+,769,0.451744,0.293278
Low-,719,-0.456006,0.266048


In [4]:
#Test your code
change_groups("AAPL.csv",1.0)

,count,mean,stdev
High+,698,2.128898,1.323162
High-,520,-2.264811,1.439012
Low+,769,0.451744,0.293278
Low-,719,-0.456006,0.266048


In [6]:
import pandas as pd
import numpy as np

<h1>Problem 2</h1>
A manufacturer has data on orders from customers and product prices in two dataframes (see below). They want to use this data to answer the following questions:
<ol>
    <li>Which customer is responsible for the most revenue</li>
    <li>Which customer is responsible for the highest profit</li>
    <li>Which product is responsible for the highest (dollar) profit</li>
    <li>Which customer and product combination is responsible for the most orders</li>
</ol>
<p>
Obviously, your code should work for any actual data values and pandas dataframes of any length!
<p>For the data below: your answers should be:
    
<pre>
Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair
</pre>

<h2>Useful functions:</h2>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html">pd.sort_values</a> </li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">pandas dataframe join</a> the last example on the linked page is probably what you need here!</li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">pd.groupby</a></li>

In [1]:
import pandas as pd
import numpy as np
order_data = pd.DataFrame([["001","010",24],
                           ["007","012",35],
                           ["001","011",15],
                           ["005","010",30],
                           ["007","011",17],
                           ["005","011",81],
                           ["001","010",32],
                           ["007","012",89],
                           ["001","010",16],
                           ["001","010",33]],columns=["customer","product","amount"])
products = pd.DataFrame([['010',22.3,17.2],
                        ['011',11.7,5.5],
                        ['012',62.5,61.4]],columns=['product','price','cost'])                     

In [2]:
joined_df = order_data.join(products.set_index('product'),on="product")
joined_df['profit'] = joined_df['amount'] * (joined_df['price'] - joined_df['cost']) 
joined_df['revenue'] = joined_df['amount'] * joined_df['price']
profits = joined_df.groupby('customer').sum()['profit']
revenues = joined_df.groupby('customer').sum()['revenue']
max_product = joined_df.groupby('product').sum()['profit']
print("Customer with most profit:",profits.sort_values(ascending=False).index[0])
print("Customer with most revenue:",revenues.sort_values(ascending=False).index[0])
print("Product with most profit:",max_product.sort_values(ascending=False).index[0])
most = order_data.groupby(['customer','product']).count().sort_values(by="amount")
print("Customer %s with product %s with %d orders is the most ordered customer product pair"%
      (most.index[-1][0],
      most.index[-1][1],
      most.amount[-1])
     )

Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair


In [4]:
most

amount
customer product        
001      011           1
005      010           1
         011           1
007      011           1
         012           2
001      010           4